In [21]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import (
    Dataset,
    load_dataset,
    load_from_disk
)

In [22]:
drug_dataset_reloaded = load_from_disk("drug-reviews") 
# drug_dataset_reloaded

In [23]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Préparation des labels : convertir les ratings en indices de classe
# Les ratings vont de 1 à 10, nous les convertissons en indices 0-9
def prepare_labels(example):
    # Convertir le rating en indice de classe (rating 1.0 -> classe 0, rating 10.0 -> classe 9)
    example['labels'] = int(example['rating']) - 1
    return example

# Appliquer la préparation des labels avant tokenisation
drug_dataset_with_labels = drug_dataset_reloaded.map(prepare_labels)

def tokenize_and_split(example):
    result = tokenizer(
        example['review'],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True
    )
    # Extraire la correspondance entre les nouveaux et les anciens indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in example.items():
        result[key] = [values[i] for i in sample_map]
    return result

tokenized_dataset = drug_dataset_with_labels.map(tokenize_and_split, batched=True)
print("Colonnes après tokenisation:", tokenized_dataset["train"].column_names)
print("Exemple de labels:", tokenized_dataset["train"][0]["labels"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 27703/27703 [00:01<00:00, 17953.81 examples/s]

Colonnes après tokenisation: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']
Exemple de labels: 1


In [25]:
# Configuration optimisée pour un entraînement rapide
training_args = TrainingArguments(
    output_dir="training_med",
    num_train_epochs=1,              # 1 époque au lieu de 3 par défaut
    per_device_train_batch_size=32,  # Batch size plus grand pour aller plus vite
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    warmup_steps=100,                # Moins de warmup steps
    max_steps=500,                   # Limiter le nombre de steps total
)

# Configurer le modèle avec le bon nombre de classes (ratings de 1 à 10 = 10 classes)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    num_labels=10  # 10 classes pour les ratings de 1 à 10
)
print(f"Modèle configuré avec {model.num_labels} classes")
print(f"Configuration d'entraînement optimisée pour ~10-15 minutes au lieu de 3h")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle configuré avec 10 classes
Configuration d'entraînement optimisée pour ~10-15 minutes au lieu de 3h


In [26]:
# Option 1: Utiliser un sous-ensemble pour un entraînement ultra-rapide (2-3 minutes)
# Décommentez ces lignes pour un test vraiment rapide
train_subset = tokenized_dataset["train"].select(range(5000))  # 5k exemples au lieu de 110k
eval_subset = tokenized_dataset["validation"].select(range(1000))  # 1k exemples au lieu de 27k

# Option 2: Utiliser le dataset complet (plus long mais meilleur résultat)
# train_subset = tokenized_dataset["train"]
# eval_subset = tokenized_dataset["validation"]

print(f"Taille du dataset d'entraînement : {len(train_subset)}")
print(f"Taille du dataset de validation : {len(eval_subset)}")

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    data_collator=data_collator,
    processing_class=tokenizer,
)

Taille du dataset d'entraînement : 5000
Taille du dataset de validation : 1000


In [27]:

print(f"Nombre de classes du modèle : {model.num_labels}")
print(f"Colonnes du dataset : {tokenized_dataset['train'].column_names}")
print(f"Exemple de labels dans le dataset : {tokenized_dataset['train'][0]['labels']}")
print(f"Type des labels : {type(tokenized_dataset['train'][0]['labels'])}")

import numpy as np
sample_labels = [tokenized_dataset['train'][i]['labels'] for i in range(100)]
unique_labels = np.unique(sample_labels)
print(f"Labels uniques dans l'échantillon : {unique_labels}")
print(f"Range des labels : min={min(sample_labels)}, max={max(sample_labels)}")

Nombre de classes du modèle : 10
Colonnes du dataset : ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']
Exemple de labels dans le dataset : 1
Type des labels : <class 'int'>
Labels uniques dans l'échantillon : [0 1 2 3 4 5 6 7 8 9]
Range des labels : min=0, max=9


In [28]:
print("Structure du dataset:")
print(drug_dataset_reloaded)
print("\nExemple du dataset d'entraînement:")
print(drug_dataset_reloaded["train"][0])
print("\nColonnes disponibles:", drug_dataset_reloaded["train"].column_names)


Structure du dataset:
DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Exemple du dataset d'entraînement:
{'patient_id': 89879, 'drugName': 'Cyclosporine', 'condition': 'keratoconjunctivitis sicca', 'review': '"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I\'ve had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put t

In [29]:
trainer.train()


Step,Training Loss,Validation Loss
200,1.767600,1.799052
400,1.348700,1.826062


/Users/nath/Desktop/huggingFace_LLM2/env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/nath/Desktop/huggingFace_LLM2/env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/nath/Desktop/huggingFace_LLM2/env/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=500, training_loss=1.648895965576172, metrics={'train_runtime': 347.6231, 'train_samples_per_second': 46.027, 'train_steps_per_second': 1.438, 'total_flos': 1047783478038528.0, 'train_loss': 1.648895965576172, 'epoch': 3.1847133757961785})

In [ ]:
# ===== ÉVALUATION ET TEST DU MODÈLE =====
# Exécutez ces cellules après l'entraînement

# 1. Évaluation sur le dataset de test
print("🔍 Évaluation du modèle sur le dataset de test...")
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"].select(range(1000)))  # 1k échantillons pour plus rapide
print("Résultats sur le dataset de test:")
for key, value in test_results.items():
    print(f"  {key}: {value:.4f}")


In [ ]:
# 2. Fonction pour faire des prédictions sur de nouveaux textes
import torch
import torch.nn.functional as F

def predict_rating(text, model, tokenizer, max_length=128):
    """
    Prédit le rating (1-10) pour un texte donné
    """
    # Tokeniser le texte
    inputs = tokenizer(
        text, 
        truncation=True, 
        max_length=max_length, 
        padding=True, 
        return_tensors="pt"
    )
    
    # Faire la prédiction
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = F.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0, predicted_class].item()
    
    # Convertir l'indice de classe en rating (0-9 -> 1-10)
    predicted_rating = predicted_class + 1
    
    return {
        'rating_prédit': predicted_rating,
        'confiance': confidence,
        'probabilités_par_classe': probabilities[0].tolist()
    }

print("✅ Fonction de prédiction créée !")


In [ ]:
# 3. Test avec des exemples concrets
print("Test du modèle avec des exemples concrets...")

# Exemples de reviews avec différents sentiments
exemples_test = [
    {
        "text": "This medication is absolutely amazing! It completely cured my condition with no side effects. I feel like a new person!",
        "sentiment_attendu": "Très positif (rating élevé)"
    },
    {
        "text": "Terrible drug, made me feel worse than before. Severe side effects and no improvement at all. Waste of money.",
        "sentiment_attendu": "Très négatif (rating faible)"
    },
    {
        "text": "The medication works okay, some improvement but also some minor side effects. Could be better.",
        "sentiment_attendu": "Neutre/moyen (rating moyen)"
    },
    {
        "text": "Good results overall. The drug helped significantly with my symptoms. Minor side effects but manageable.",
        "sentiment_attendu": "Positif (rating bon)"
    }
]

print("Prédictions sur les exemples de test:")
print("=" * 80)

for i, exemple in enumerate(exemples_test, 1):
    print(f"\n Exemple {i}:")
    print(f"Texte: {exemple['text']}")
    print(f"Sentiment attendu: {exemple['sentiment_attendu']}")
    
    prediction = predict_rating(exemple['text'], model, tokenizer)
    print(f"Rating prédit: {prediction['rating_prédit']}/10")
    print(f"Confiance: {prediction['confiance']:.3f}")
    print("-" * 40)


In [ ]:
# 4. Analyse détaillée des performances
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

def evaluate_detailed(model, tokenizer, test_dataset, max_samples=1000):
    """
    Évaluation détaillée avec métriques complètes
    """
    print(" Analyse détaillée des performances...")
    
    # Prendre un échantillon du dataset de test
    test_sample = test_dataset.select(range(min(max_samples, len(test_dataset))))
    
    # Prédictions
    predictions = []
    true_labels = []
    
    for example in test_sample:
        pred = predict_rating(example['review'], model, tokenizer)
        predictions.append(pred['rating_prédit'] - 1)  # Convertir en 0-9
        true_labels.append(example['labels'])
    
    # Calcul des métriques
    accuracy = accuracy_score(true_labels, predictions)
    
    print(f" Résultats sur {len(test_sample)} échantillons:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Accuracy en %: {accuracy*100:.2f}%")
    
    # Classification report
    print("\n Rapport de classification détaillé:")
    class_names = [f"Rating {i+1}" for i in range(10)]
    print(classification_report(true_labels, predictions, 
                              target_names=class_names, 
                              zero_division=0))
    
    return accuracy, predictions, true_labels

# Exécuter l'évaluation détaillée (à faire après l'entraînement)
# accuracy, preds, labels = evaluate_detailed(model, tokenizer, tokenized_dataset["test"])


In [32]:
# 5. Test interactif - entrez votre propre texte !
def test_interactif():
    """
    Fonction pour tester le modèle avec vos propres textes
    """
    print(" Test interactif du modèle")
    print("Entrez vos propres reviews de médicaments pour voir les prédictions!")
    print("(Tapez 'quit' pour arrêter)")
    print("-" * 60)
    
    while True:
        user_input = input("\n Entrez votre review: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print(" Aurevoir !")
            break
            
        if user_input.strip():
            try:
                prediction = predict_rating(user_input, model, tokenizer)
                print(f" Rating prédit: {prediction['rating_prédit']}/10")
                print(f" Confiance: {prediction['confiance']:.3f}")
                
                # Interpretation du rating
                rating = prediction['rating_prédit']
                if rating >= 8:
                    interpretation = "😍 Très satisfait"
                elif rating >= 6:
                    interpretation = "😊 Satisfait" 
                elif rating >= 4:
                    interpretation = "😐 Neutre"
                else:
                    interpretation = "😞 Pas satisfait"
                    
                print(f"Interprétation: {interpretation}")
                
            except Exception as e:
                print(f" Erreur: {e}")
        else:
            print("Veuillez entrer un texte non vide")

# Pour lancer le test interactif, décommentez la ligne suivante après l'entraînement:
test_interactif() 


 Test interactif du modèle
Entrez vos propres reviews de médicaments pour voir les prédictions!
(Tapez 'quit' pour arrêter)
------------------------------------------------------------
 Erreur: name 'predict_rating' is not defined
 Erreur: name 'predict_rating' is not defined
 Aurevoir !
